In [12]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [13]:
import sys
sys.path.append('../src')

from models.classification_methods import get_classification_report

In [14]:
test_results_path = '../reports/test_results/'

list_df_t = os.listdir(test_results_path)
list_df_t.sort()
list_df_t

['DummyClassifier_bo_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_bo_users_Stance_test_results.csv',
 'DummyClassifier_bo_users_Timeline_test_results.csv',
 'DummyClassifier_cl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_cl_users_Stance_test_results.csv',
 'DummyClassifier_cl_users_Timeline_test_results.csv',
 'DummyClassifier_co_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_co_users_Stance_test_results.csv',
 'DummyClassifier_co_users_Timeline_test_results.csv',
 'DummyClassifier_gl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_gl_users_Stance_test_results.csv',
 'DummyClassifier_gl_users_Timeline_test_results.csv',
 'DummyClassifier_ig_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_ig_users_Stance_test_results.csv',
 'DummyClassifier_ig_users_Timeline_test_results.csv',
 'DummyClassifier_lu_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_lu_users_Stance_test_r

In [15]:
# Target list
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
]

dict_cp = {
    'cl':'Hidroxicloroquina',
    'lu':'Lula',
    'co':'Sinovac',
    'ig':'Church',
    'gl':'Globo TV',
    'bo':'Bolsonaro',
}

names = list(dict_cp.values())
names

['Hidroxicloroquina', 'Lula', 'Sinovac', 'Church', 'Globo TV', 'Bolsonaro']

## Create complete table

In [16]:
# (vectorizer,estimator, path_sring) 
results_tuples_stance = [
    # Stance
    ("Stance", "-" ,"DummyClassifier", "DummyClassifier_{target}_users_Stance_test_results.csv"),
    ("Stance", "TF-IDF" ,"xgboost", "XGBClassifier_TfidfVectorizer_{target}_users_Stance_test_results.csv"),
    ("Stance", "bertabaporu-base-uncased" ,"xgboost", "bertimbau_xgb_{target}_users_emb_Stance_test_results.csv"),
    ("Stance", "-" ,"bertabaporu-base-uncased", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv"),
    ("Stance", "-",  "llama3:7b", "llama3_{target}_Stance_prompt2_Stance_test_results.csv"),
    
    # Texts
    ("Texts", "-" ,"DummyClassifier", "DummyClassifier_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "TF-IDF" ,"xgboost", "XGBClassifier_TfidfVectorizer_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "bertabaporu-base-uncased" ,"xgboost", "bertimbau_xgb_{target}_top_mentioned_timelines_emb_Texts_test_results.csv"),
    
    # Timeline
    ("Timeline", "-" ,"DummyClassifier", "DummyClassifier_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "TF-IDF" ,"xgboost", "XGBClassifier_TfidfVectorizer_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "bertabaporu-base-uncased" ,"xgboost", "bertimbau_xgb_{target}_users_emb_Timeline_test_results.csv")
    
]

list_results = []
for text_col, vectorizer, estimator, path_results in results_tuples_stance:
    
    list_cr = []
    
    for target in target_list:
        
        
        path = test_results_path + path_results.format(target = target)
        df_results = pd.read_csv(path)
        df_results_or = df_results.copy()
        
        # get classification report df
        df_classification_report = get_classification_report(df_results.test, df_results.pred, cr_args = {})
        
        # create multindex
        column_indexes = [(metric,dict_cp[target]) for metric in df_classification_report.columns]
        multi_index_cols = pd.MultiIndex.from_tuples(column_indexes, names=['metric', 'target'])
        rows_indexes = [(text_col, vectorizer, estimator, cl) for cl in df_classification_report.index]
        multi_index_rows = pd.MultiIndex.from_tuples(rows_indexes, names=['text_col','vectorizer', 'estimator', 'class'])
        df_classification_report.columns = multi_index_cols
        df_classification_report.index = multi_index_rows
        
        # print(text_col, vectorizer, estimator,target)
        # print(path)
        # display(df_classification_report)
        
        list_cr.append(df_classification_report)
        
    df_results = pd.concat(list_cr, axis = 1)
    
    list_results.append(df_results)
    
df_results_final = pd.concat(list_results)

In [17]:
df_results_final

metric                                                                  precision  \
target                                                                     Church   
text_col vectorizer               estimator                class                    
Stance   -                        DummyClassifier          against       0.565943   
                                                           accuracy      0.565943   
                                                           weighted avg  0.320292   
                                                           macro avg     0.282972   
                                                           for           0.000000   
         TF-IDF                   xgboost                  against       0.732394   
                                                           accuracy      0.709516   
                                                           weighted avg  0.708016   
                                                           macro avg     0.704312   
                                                           for           0.676230   
         bertabaporu-base-uncased xgboost                  against       0.834646   
                                                           accuracy      0.859766   
                                                           weighted avg  0.864606   
                                                           macro avg     0.869158   
                                                           for           0.903670   
         -                        bertabaporu-base-uncased against       0.850267   
                                                           accuracy      0.871452   
                                                           weighted avg  0.874748   
                                                           macro avg     0.878467   
                                                           for           0.906667   
                                  llama3:7b                for           0.640000   
                                                           accuracy      0.729549   
                                                           macro avg     0.747711   
                                                           weighted avg  0.761916   
                                                           against       0.855422   
Texts    -                        DummyClassifier          against       0.565943   
                                                           accuracy      0.565943   
                                                           weighted avg  0.320292   
                                                           macro avg     0.282972   
                                                           for           0.000000   
         TF-IDF                   xgboost                  against       0.638677   
                                                           accuracy      0.616027   
                                                           weighted avg  0.610089   
                                                           macro avg     0.605746   
                                                           for           0.572816   
         bertabaporu-base-uncased xgboost                  against       0.639474   
                                                           accuracy      0.611018   
                                                           weighted avg  0.605691   
                                                           macro avg     0.600559   
                                                           for           0.561644   
Timeline -                        DummyClassifier          against       0.565943   
                                                           accuracy      0.565943   
                                                           weighted avg  0.320292   
                                                           macro avg     0.282972   
     

In [18]:
df_results_final.to_excel("../reports/table_complete_results.xlsx")

## Create table f1 macro

In [19]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "macro avg" in col else False for col in df_results_final.index]

f1_macro_df = df_results_final.loc[mask_macro,mask_f1]
f1_macro_df[('f1-score','overall')] = f1_macro_df.mean(axis=1)

f1_macro_df

metric                                                                f1-score  \
target                                                                  Church   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.361407   
         TF-IDF                   xgboost                  macro avg  0.702021   
         bertabaporu-base-uncased xgboost                  macro avg  0.853801   
         -                        bertabaporu-base-uncased macro avg  0.866621   
                                  llama3:7b                macro avg  0.729458   
Texts    -                        DummyClassifier          macro avg  0.361407   
         TF-IDF                   xgboost                  macro avg  0.596115   
         bertabaporu-base-uncased xgboost                  macro avg  0.594754   
Timeline -                        DummyClassifier          macro avg  0.361407   
         TF-IDF                   xgboost                  macro avg  0.699122   
         bertabaporu-base-uncased xgboost                  macro avg  0.661773   

metric                                                                          \
target                                                               Bolsonaro   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.462857   
         TF-IDF                   xgboost                  macro avg  0.595312   
         bertabaporu-base-uncased xgboost                  macro avg  0.625289   
         -                        bertabaporu-base-uncased macro avg  0.739125   
                                  llama3:7b                macro avg  0.462857   
Texts    -                        DummyClassifier          macro avg  0.462857   
         TF-IDF                   xgboost                  macro avg  0.501220   
         bertabaporu-base-uncased xgboost                  macro avg  0.495578   
Timeline -                        DummyClassifier          macro avg  0.462857   
         TF-IDF                   xgboost                  macro avg  0.728220   
         bertabaporu-base-uncased xgboost                  macro avg  0.692262   

metric                                                                                  \
target                                                               Hidroxicloroquina   
text_col vectorizer               estimator                class                         
Stance   -                        DummyClassifier          macro avg          0.334878   
         TF-IDF                   xgboost                  macro avg          0.734366   
         bertabaporu-base-uncased xgboost                  macro avg          0.830948   
         -                        bertabaporu-base-uncased macro avg          0.844778   
                                  llama3:7b                macro avg          0.638889   
Texts    -                        DummyClassifier          macro avg          0.334878   
         TF-IDF                   xgboost                  macro avg          0.603727   
         bertabaporu-base-uncased xgboost                  macro avg          0.609524   
Timeline -                        DummyClassifier          macro avg          0.334878   
         TF-IDF                   xgboost                  macro avg          0.898944   
         bertabaporu-base-uncased xgboost                  macro avg          0.776979   

metric                                                                          \
target                                                                 Sinovac   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.351759   
         TF-IDF                   xgboost                  macro avg  0.752640   
         bertabaporu-base-uncas

In [20]:
f1_macro_df.to_excel("~/teste.xlsx")

In [21]:
f1_macro_df.sort_values(('f1-score','overall'))

metric                                                                f1-score  \
target                                                                  Church   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.361407   
Texts    -                        DummyClassifier          macro avg  0.361407   
Timeline -                        DummyClassifier          macro avg  0.361407   
Texts    bertabaporu-base-uncased xgboost                  macro avg  0.594754   
         TF-IDF                   xgboost                  macro avg  0.596115   
Stance   -                        llama3:7b                macro avg  0.729458   
         TF-IDF                   xgboost                  macro avg  0.702021   
Timeline bertabaporu-base-uncased xgboost                  macro avg  0.661773   
         TF-IDF                   xgboost                  macro avg  0.699122   
Stance   bertabaporu-base-uncased xgboost                  macro avg  0.853801   
         -                        bertabaporu-base-uncased macro avg  0.866621   

metric                                                                          \
target                                                               Bolsonaro   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.462857   
Texts    -                        DummyClassifier          macro avg  0.462857   
Timeline -                        DummyClassifier          macro avg  0.462857   
Texts    bertabaporu-base-uncased xgboost                  macro avg  0.495578   
         TF-IDF                   xgboost                  macro avg  0.501220   
Stance   -                        llama3:7b                macro avg  0.462857   
         TF-IDF                   xgboost                  macro avg  0.595312   
Timeline bertabaporu-base-uncased xgboost                  macro avg  0.692262   
         TF-IDF                   xgboost                  macro avg  0.728220   
Stance   bertabaporu-base-uncased xgboost                  macro avg  0.625289   
         -                        bertabaporu-base-uncased macro avg  0.739125   

metric                                                                                  \
target                                                               Hidroxicloroquina   
text_col vectorizer               estimator                class                         
Stance   -                        DummyClassifier          macro avg          0.334878   
Texts    -                        DummyClassifier          macro avg          0.334878   
Timeline -                        DummyClassifier          macro avg          0.334878   
Texts    bertabaporu-base-uncased xgboost                  macro avg          0.609524   
         TF-IDF                   xgboost                  macro avg          0.603727   
Stance   -                        llama3:7b                macro avg          0.638889   
         TF-IDF                   xgboost                  macro avg          0.734366   
Timeline bertabaporu-base-uncased xgboost                  macro avg          0.776979   
         TF-IDF                   xgboost                  macro avg          0.898944   
Stance   bertabaporu-base-uncased xgboost                  macro avg          0.830948   
         -                        bertabaporu-base-uncased macro avg          0.844778   

metric                                                                          \
target                                                                 Sinovac   
text_col vectorizer               estimator                class                 
Stance   -                        DummyClassifier          macro avg  0.351759   
Texts    -                        DummyClassifier          macro avg  0.351759   
Timeline -                     